In [6]:
# Install needed lib
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [7]:
import os
import torch
from datasets import load_dataset
import datasets
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    HfArgumentParser, TrainingArguments, pipeline, logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
# from trl.utils import get_peft_config

In [8]:
# # Checking and formating data set
# # Org: https://huggingface.co/datasets/timdettmers/openassistant-guanaco
# # Template: https://huggingface.co/datasets/mlabonne/guanaco-llama2?row=0
# # Our: https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k

# from datasets import load_dataset
# import re
# # Load the dataset
# oa_dataset = load_dataset('timdettmers/openassistant-guanaco')

# # Shuffle the dataset and slice it
# oa_dataset = oa_dataset['train'].shuffle(seed=42).select(range(1000))

# # Define a function to transform the data
# def transform_conversation(example):
#     convo_text = example['text']
#     segments = convo_text.split('###')

#     reformatted_segments = []

#     for i in range(1, len(segments) - 1, 2):
#         user_text = segments[i].strip().replace('Human:', '').strip()
#         ai_text = segments[i+1].strip().replace('Assistant:', '').strip() if i + 1 < len(segments) else ''
#         reformatted_segments.append(f'<s>[INST] {user_text} [/INST] {ai_text} </s>')

#     return {'text': ''.join(reformatted_segments)}



# # Apply the transformation
# transformed_dataset = oa_dataset.map(transform_conversation)


In [9]:
# def preprocess_function(example):
#     # T5 format
#     question = example['question'].strip()
#     context = example['context'].strip()
#     # answer = example['answer'].strip()

#     input_text = f"<s>[INST] {question} [/INST] {context} </s>"

#     return {
#         "text": input_text
#         # "output": answer
#     }

def preprocess_function(example):
    input_text = f"""Context: {example['context']}
Question: {example['question']}
Answer: {example['answer']}"""

    return {
        "text": input_text
    }
import json
# data_files = {"train": "dialy_qa_pairs.json"}
# dataset = load_dataset("json", data_files=data_files)
# transformed_dataset = dataset.map(preprocess_function)
with open("lichsu_qa_pairs_v2.json", "r", encoding="utf-8") as f:
    data = json.load(f)
with open("lichsu_qa_pairs (3).json", "r", encoding="utf-8") as f:
    data = json.load(f)
with open("dialy_qa_pairs.json", "r", encoding="utf-8") as f:
    data += json.load(f)

with open("Dia_Ly_12_canh_dieu_qa_pairs_1.json", "r", encoding="utf-8") as f:
    data += json.load(f)

with open("Giao_duc_cong_dan_12_canh_dieu_qa_pairs.json", "r", encoding="utf-8") as f:
    data += json.load(f)

with open ("Giao_duc_cong_dan_12_qa_pairs.json", "r", encoding="utf-8") as f:
    data += json.load(f)
dataset = datasets.Dataset.from_list(data)

transformed_dataset = dataset.map(preprocess_function).select_columns(['text'])
train_test_split = transformed_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

Map:   0%|          | 0/3778 [00:00<?, ? examples/s]

In [10]:
# from datasets import load_dataset

# # Load a local CSV file
# dataset = load_dataset('csv', data_files='path/to/your/localfile.csv')

# # Preview the dataset
# print(dataset)

# data_files = {"train": "train.csv", "test": "test.csv"}
# dataset = load_dataset("namespace/your_dataset_name", data_files=data_files)


In [11]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"
finetune_model = "Llama-2-7b-chat-finetune"


# Output folder
output_dir = "./Llama-2-7b-chat-finetune"

# No of epochs
num_train_epochs =1

# No change params
use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant = True, "float16", "nf4", False # To quantization
lora_r, lora_alpha, lora_dropout = 64, 16, 0.1
fp16, bf16 =  False, False
per_device_train_batch_size, per_device_eval_batch_size = 4, 4
gradient_accumulation_steps, gradient_checkpointing, max_grad_norm = 1, True, 0.3
learning_rate, weight_decay, optim = 2e-4, 0.001, "paged_adamw_32bit"
lr_scheduler_type, max_steps, warmup_ratio = "cosine", -1, 0.03
group_by_length, save_steps, logging_steps = True, 0, 25
max_seq_length, packing, device_map = None, False, {"": 0}


In [12]:
# transformed_dataset is there

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model, # model se fintune
    train_dataset=train_dataset, # Dataset
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3022 [00:00<?, ? examples/s]

Map:   0%|          | 0/756 [00:00<?, ? examples/s]

In [24]:
trainer.train()
trainer.model.save_pretrained(finetune_model)

NameError: name 'trainer' is not defined

In [23]:
# Test model
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
# prompt = "Nhiệt độ trung bình cận xích đạo"
#question = "Nhiệt độ trung bình cận xích đạo?"
#context = "- Nền nhiệt độ thiên về khí hậu cận xích đạo, quanh năm nóng, nhiệt độ trung bình năm trên 25°C và không có tháng nào dưới 20°C."
# question = "Vùng tiếp giáp lãnh hải rộng bao nhiêu hải lý?"
# context = "Vùng tiếp giáp lãnh hải của nước ta rộng 12 hải lí ."
# question ="Tại sao biển đông lại quan trọng?"
# context = "Biển Đông đối với nước ta là một hướng chiến lược quan trọng trong công cuộc xây dựng phát triển kinh tế và bảo vệ đất nước."
# question = "Nhiệt độ trung bình của khu vực ôn đới là bao nhiêu?"
# context = "- Ở khu vực ôn đới, nhiệt độ trung bình năm dao động từ 10°C đến 20°C, có sự phân chia rõ ràng giữa các mùa xuân, hạ, thu, đông."

# question ="Tính chất khí hậu của miền bắc bộ là gì?"
# context ="- Khí hậu có tính chất khí hậu ôn đới, quanh năm nhiệt độ dưới 15°C, mùa đông xuống dưới 5°C; có các loài thực vật ôn đới như dừa quyến, lãnh sam, thiết sam."
# lịch sử
question = "Chiến tranh thế giới thứ hai kết thúc khi nào?"
# context = "- Cuộc cách mạng công nghiệp bắt đầu từ thế kỷ 18 ở Anh, với sự xuất hiện của máy móc như máy hơi nước và sự phát triển của ngành dệt may. Điều này đã dẫn đến sự thay đổi lớn trong kinh tế và xã hội."
test_prompt = f"""
Question: {question}"""
pipe = pipeline(task="text2text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(f"[INST] {test_prompt} [/INST]")
result = pipe(test_prompt)
print(result[0]['generated_text'])

NameError: name 'model' is not defined

In [ ]:
# from transformers import pipeline

# # Định nghĩa mô hình và tokenizer
# # model = ...
# # tokenizer = ...
# from peft import PeftModel
# from transformers import AutoModelForCausalLM

# base_model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
# model = PeftModel.from_pretrained(base_model, "duyan2803/llama-chat-finetuned")
# tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
# # Khởi tạo pipeline
# pipe = pipeline(task="question-answering", model=model, tokenizer=tokenizer, max_length=200)

# # Định nghĩa input đúng định dạng
# test_prompt = {
#     "question": "Thời tiết ở cận xích đạo như thế nào?",
#     "context": "Cận xích đạo thường có thời tiết nóng ẩm quanh năm, với lượng mưa lớn và ít sự chênh lệch nhiệt độ giữa các mùa."
# }

# # Gọi pipeline
# result = pipe(test_prompt)

# # In kết quả
# print(result)


In [26]:
# # Free VRAM
import gc
del base_model
del model, pipe, trainer

gc.collect()

NameError: name 'model' is not defined

In [27]:
# # Reload and merge
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, finetune_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)